## We start with Importing the necessary libraries

In [ ]:
import pandas as pd
import sqlite3

## SQL Hepler Functions (Note: You dont need to worry about this part)

In [ ]:
import chardet

file  = '/content/northwind_with_headers.csv'
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

In [ ]:
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table

    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()

def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe

    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.

    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

## Execute Query

In [ ]:
input_df = pd.read_csv('/content/northwind_with_headers.csv', encoding = 'ISO-8859-1')

pd_to_sqlDB(input_df,
            table_name='Customers',
            db_name='default.db')

## Doing the very Basic - Select Everyting 

In [ ]:
sql_query_string = """
    SELECT * FROM Customers
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## Limiting the number of rows you want to see

In [ ]:
sql_query_string = """
    SELECT * FROM Customers limit 5
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## Viewing certain Columns  

In [ ]:
sql_query_string = """
    SELECT CustomerID, ContactName, Address, Postalcode FROM Customers limit 7
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## Viewing the distinct values in a column.
#### The SQL select distinct Syntax - 

SELECT DISTINCT col1, col2, ... FROM table_name;

In [ ]:
sql_query_string = """
    Select Distinct ContactTitle FROM Customers
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

In [ ]:
sql_query_string = """
    Select DISTINCT CustomerID, ContactTitle FROM Customers
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## Counting the number of items in a colums

Syntax: SELECT count(col1) from table_name

In [ ]:
sql_query_string = """
    Select count(ContactTitle) FROM Customers
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

In [ ]:
sql_query_string = """
    Select count(Distinct ContactTitle) FROM Customers
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## The sql WHERE clause

Syntax: SELECT *col1, col2,..* FROM *table_name* WHERE *condition*;


In [ ]:
sql_query_string = """
    SELECT * FROM Customers WHERE Country='Mexico';
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df


### Combining Conditions using AND



In [ ]:
sql_query_string = """
    SELECT * FROM Customers WHERE Country='Mexico' AND ContactTitle = 'Owner' ;
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## We can use *Where* Clause with different *operators* to build conditions and access the data


Equal: `=`

Greater  than: `>`

 Less than: `<`

Greater than or equal to: `>=`

Less than or equal to: `<=`

Not equal to : `<>` or `!=`

Between a certain range: `BETWEEN`

To search for a pattern:  `LIKE`

In [ ]:
sql_query_string = """
    SELECT * FROM Customers WHERE Country='Mexico';
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

In [ ]:
sql_query_string = """
    SELECT * FROM Customers WHERE PostalCode > '5021' limit 5;
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

In [ ]:
sql_query_string = """
    SELECT * FROM Customers WHERE PostalCode > '5021' limit 5;
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

In [ ]:
sql_query_string = """
    SELECT * FROM Customers WHERE PostalCode >= '5033' limit 5;
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

In [ ]:
sql_query_string = """
    SELECT * FROM Customers WHERE PostalCode Between '5021' AND '5033' limit 5;
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

In [ ]:
sql_query_string = """
    SELECT * FROM Customers WHERE City Like 'V%';
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## The sql ORDER BY statement

Syntax: SELECT *col1, col2, ...* FROM *table_name* ORDER BY *col1, col2, ... ASC|DESC*;

## Accessing the tables in ascending order to the Country

In [ ]:
sql_query_string = """
    SELECT * FROM Customers ORDER BY Country limit 5;
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## Accessing the tables in descending order to the Country


In [ ]:
sql_query_string = """
    SELECT * FROM Customers ORDER BY Country DESC limit 5;
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## combining the ASC and DESC commands

In [ ]:
sql_query_string = """
    SELECT * FROM Customers ORDER BY Country ASC, CustomerID DESC limit 5;
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## The SQL Null Values

syntax: SELECT *column_names* FROM *table_name* WHERE *column_name* IS NULL;

In [ ]:
sql_query_string = """
    SELECT CustomerID, ContactName, Address FROM Customers WHERE Region IS NULL limit 5;
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## The SQL NOT Null Values

syntax: SELECT *column_names* FROM *table_name* WHERE *column_name* IS not NULL;

In [ ]:
sql_query_string = """
    SELECT CustomerID, ContactName, Address FROM Customers WHERE Region IS not NULL limit 5;
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

## The sql INSERT Statement

Syntax: INSERT INTO *table_name (column1, column2, column3, ...)*

VALUES *(value1, value2, value3, ...);*



This is one of the thing we cannot show in the colab notebook due to implementation isssue

In [ ]:
sql_query_string = """
    INSERT INTO Customers (CustomerID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax)
VALUES ('COSMOS', 'Starry night', 'Erichsen', 'Admin', 'Av. Del 9540 60-C', 'Brooklyn', 'Kings', '11450', 'USA', '2324-3453', '454-4543');
"""
 
# Step 4: Exectue the SQL query
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

### There are a lot more commands in SQL you can use to manipulate and access and update databases and tables, however, The colab limits to mostly manipulate and access but no write an insert command also the database is pretty basic in terms of operations that can be performed. An actual datatable in real-world example is a lot more messy and complicated and large.

### Below is the list of commands you should practice in order to have understinding or the most basic SQL commands 

Resource: 

[SQL PAd](https://sqlpad.io)

[SQL Leetcode](https://leetcode.com/problemset/database/) (For Data Engineering Roles and Full Stack Engineering interviews)



### Here you can practice a lot of SQL without going through the pain of installling SQL server, Editor and The Database.

## The SQL UPDATE Statement

Syntax: UPDATE *table_name*
SET *column1 = value1, column2 = value2, ...*
WHERE *condition*;

## The SQL DELETE Statement

Syntax: DELETE FROM *table_name* WHERE *condition*;


# THe SQL Mathematical Operations to access the Data


## The SQL MIN() function

Syntax: SELECT MIN(*column_name*)
FROM *table_name*
WHERE *condition;


## The SQL MAX() Function

Syntax: SELECT MAX(*column_name*)
FROM *table_name*
WHERE *condition*;

## The COUNT() Function

Syntax: SELECT COUNT(*column_name*)
FROM *table_name*
WHERE *condition*;

## The SQL AVG() Function

Syntax: SELECT AVG(column_name) FROM table_name WHERE condition;

## The SQL SUM() Function

Syntax: SELECT SUM(column_name) FROM table_name WHERE condition;

## Advance SQL Commands

Everything we saw till now was related to a solo tables, however a database can contain multiple table and you may requre to access a combined data from both the tables.

SQL JOIN: A JOIN clause is used to combine rows from two or more tables, based on a related column between them.

Different Types of SQL JOINs
Here are the different types of the JOINs in SQL:


1.   (INNER) JOIN: Returns records that have matching values in both tables
2.   LEFT (OUTER) JOIN: Returns all records from the left table, and the matched records from the right table
3. RIGHT (OUTER) JOIN: Returns all records from the right table, and the matched records from the left table
4. FULL (OUTER) JOIN: Returns all records when there is a match in either left or right table